In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# hyperparamters
days = 365
gamma = 0.9
bank = 0
reward_rent = 10
reward_empty = 0
transfer_cost = 2
theta = 0.1
max_cars = 20

In [ ]:
# environment class
class Location():
    
    def __init__(self, name, cars, exp_out, exp_in):
        self.name = name
        self.cars = cars
        self.exp_out = exp_out
        self.exp_in = exp_in
        
    def move(self, location, cars):
        self.cars = self.cars-cars
        location.cars = np.clip(location.cars+cars, 1, 20)
        
    def rent(self, cars):
        self.cars = self.cars-min(self.cars, cars)
        return 10 * min(self.cars, cars)
    
    def restore(self, cars):
        self.cars = np.clip(self.cars+cars, 1, 20)
        
    def episode(self):
        return np.random.poisson(self.exp_in), np.random.poisson(self.exp_out)
    
    def rental_likelihood(self, x):
        return np.exp(-self.exp_out) * pow(self.exp_out, x) / np.math.factorial(x)
    
    def restore_likelihood(self, x):
        return np.exp(-self.exp_in) * pow(self.exp_in, x) / np.math.factorial(x)

In [ ]:
# map (s,a) to (s')
def next_state(state, action):
    
    if action > 0:
        s1 = min(state[0] - min(np.abs(action), state[0]), max_cars+5)
        s2 = min(state[1] + min(np.abs(action), state[0]), max_cars+5)
        return tuple((s1,s2)), min(np.abs(action), state[0])
    elif action < 0:
        s1 = min(state[0] + min(np.abs(action), state[1]), max_cars+5)
        s2 = min(state[1] - min(np.abs(action), state[1]), max_cars+5)
        return tuple((s1,s2)), min(np.abs(action), state[1])
    else:
        return state, 0
    
# run policy evaluation on state_value
def policy_evaluation():
    while True:
        delta = 0
        for state in tqdm(state_value.keys()):
            old = state_value[state]
            state_value[state] = expected_reward(state, policy[state])
            delta = max(delta, np.abs(old-state_value[state]))
        print(delta)
        if delta < theta:
            break
            
# run policy imporvement on state_value and policy
def policy_improvement():
    policy_stable = True
    for state in tqdm(state_value.keys()):
        old = policy[state]
        policy[state] = max(A, key=lambda x: expected_reward(state, x))
        if old != policy[state]:
            policy_stable = False
    return policy_stable

# get the expected reward from a state action pair            
def expected_reward(state, action):
    newstate, moved = next_state(state, action)
    reward = 0
    sum_prob = 0
    # only use in range 0-6 for rentals and returns for compute
    # i argue this number is okay since we cover 99.9+ percent of the probability distribution
    for loc1_rentals in range(6):
        for loc2_rentals in range(6):
            for loc1_returns in range(6):
                for loc2_returns in range(6):
                    probability = (
                        loc1.rental_likelihood(loc1_rentals) *
                        loc2.rental_likelihood(loc2_rentals) *
                        loc1.restore_likelihood(loc1_returns) *
                        loc2.restore_likelihood(loc2_returns)
                    )                
                    loc1_cars = np.clip(newstate[0] - loc1_rentals + loc1_returns, 0, max_cars)
                    loc2_cars = np.clip(newstate[1] - loc2_rentals + loc2_returns, 0, max_cars)
                    reward += probability * (
                        -2 * np.abs(moved) + # moving penalty
                        10 * (min(loc1_rentals, newstate[0]) + min(loc2_rentals, newstate[1])) # rental reward
                        + state_value[(loc1_cars, loc2_cars)] # state-value of s'
                    )
                    
                    # part b first change
                    if action < 0:
                        reward += probability*2
                    # part b second change
                    for cars in state:
                        if cars > 10:
                            reward += probability * -4
                            break
    return reward

In [ ]:
# actions (- is loc2 --> loc1 and + is loc1 --> loc2)
A = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]

# state-value function
state_value = {
    k:v for k,v in zip(
        ((first, second) for first in range(max_cars+1) for second in range(max_cars+1)), 
        np.random.rand((max_cars+1)**2)
    )
}

# policy
policy = {
    k:v for k,v in zip(
        ((first, second) for first in range(max_cars+1) for second in range(max_cars+1)), 
        (np.random.choice(A) for choice in range((max_cars+1)**2))
    )
}

# define locations
loc1 = Location("San Diego", 20, 3, 3)
loc2 = Location("New York", 20, 4, 2)

# plot the initial state_value policy grid
plt.imshow(np.array(list(policy.values())).reshape(max_cars+1,max_cars+1))
plt.colorbar()
plt.show()
plt.clf()

# iterate for days or break when stable
# plot the state_value function every step
for day in range(days):
    policy_evaluation()
    stable = policy_improvement()
    plt.imshow(np.array(list(policy.values())).reshape(max_cars+1,max_cars+1))
    plt.colorbar()
    plt.show()
    plt.clf()
    if stable:
        break

In [ ]:
# plot the 3d gradient landscape
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X = np.arange(0, 21, 1)
Y = np.arange(0, 21, 1)
X, Y = np.meshgrid(X, Y)
Z = np.array(list(policy.values())).reshape(max_cars+1,max_cars+1)
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(-5, 5)
ax.zaxis.set_major_locator(LinearLocator(10))
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()